# Investigating the use of Hive Moderation API

Detecting cyberbullying, abusive language, threats of violence and hate speech in online conversations.

https://hivemoderation.com/text-moderation

https://docs.thehive.ai/docs/text-moderation-api

In [1]:
import os
import requests
import pandas as pd

In [2]:
API_KEY = os.environ.get("HIVE_API_KEY")

In [3]:
def classify_text(input_text):
    response = requests.post(
        url='https://api.thehive.ai/api/v2/task/sync',
        headers={'Authorization': f'Token {API_KEY}'},
        data={'text_data': input_text},
    )
    response_dict = response.json()
    
    res = response_dict['status'][0]
    pred_status = res['status']
    if not pred_status['message'] == 'SUCCESS':
        print(res)
    else:
        ner_result = res['response']['text_filters']
        lang = res['response']['language']
        predictions = [cl for cl in res['response']['output'][0]['classes'] if cl['score'] > 0]
        # Return list of predictions
        return predictions

    return []

## Predict Data

In [4]:
# Read Data
df = pd.read_csv("safebear_data.csv")
df.label = df.label.replace({
    "Racisme":"hate",
    "Insulte (et Discrimination)":"bullying",
    "Obscénité":"sexual",
    "Sexualité explicite":"sexual",
    "Menace":"violence",
    "Toxicité":"bullying",
    "Sexisme":"hate"
})
print("Classes \n",df.label.unique())

Classes 
 ['hate' 'bullying' 'sexual' 'violence']


In [5]:
predictions = []
predictions_binary = []
for i,row in df.iterrows():
    pred = classify_text(row.text)
    predictions.append(pred)
    # If any labels is predicted say its "Toxic" as binary.
    binary_label = "Toxic" if len(pred) > 0 else "Normal"
    predictions_binary.append(binary_label)

df['prediction'] = predictions
df['prediction_binary'] = predictions_binary
df['correct_multilabel'] = df.apply(lambda x: x['label'] in [cl['class'] for cl in x.prediction],axis=1)
df['correct_binary'] = (df.prediction_binary == 'Toxic')

## Show Results

In [6]:
print("Binary\n==============================")
accuracy = df['correct_binary'].sum() / len(df) * 100
print(f"Correct Prediction / Total Item: {df['correct_binary'].sum()} / {len(df)}")
print(f"Accuracy: {accuracy:.2f}%")
print("")
print("Multi Label\n==============================")
accuracy = df['correct_multilabel'].sum() / len(df) * 100
print(f"Correct Prediction / Total Item: {df['correct_multilabel'].sum()} / {len(df)}")
print(f"Accuracy: {accuracy:.2f}%")

Binary
Correct Prediction / Total Item: 28 / 37
Accuracy: 75.68%

Multi Label
Correct Prediction / Total Item: 18 / 37
Accuracy: 48.65%


In [7]:
for i,row in df.iterrows():
    sign = "✅" if row.correct_binary else "❌"
    print(f"Text: {row.text}")
    print(f"Prediction: {row.prediction}")    
    print(f"Binary Toxic : {sign}")
    print(f"True Label : {row.label}\n")

Text: De toute façon t’es qu’un sale juif t’aurais du mourir en 45
Prediction: [{'class': 'hate', 'score': 2}, {'class': 'violence', 'score': 2}, {'class': 'bullying', 'score': 2}]
Binary Toxic : ✅
True Label : hate

Text: Au pire retourne à la mosquée sale arabe
Prediction: [{'class': 'hate', 'score': 2}, {'class': 'bullying', 'score': 2}]
Binary Toxic : ✅
True Label : hate

Text: C’est pas l’heure d’aller bouffer des chenilles avec tes 29 femmes le négro ?
Prediction: [{'class': 'bullying', 'score': 3}]
Binary Toxic : ✅
True Label : hate

Text: Oh mon Dieu, je n’avais pas vu un Juif courir comme ça depuis les soldes du salon du cuir
Prediction: []
Binary Toxic : ❌
True Label : hate

Text: Mais nan j’irai pas en enfer j’suis pas black
Prediction: [{'class': 'hate', 'score': 2}, {'class': 'bullying', 'score': 1}]
Binary Toxic : ✅
True Label : hate

Text: Pour vous les juifs le paradis c’est de trouver un cheveu dans votre soupe au resto pour vous faire rembourser
Prediction: []
Binary 

# Custom Sentences: Neutral Performance

In [8]:
def evaluate_sentences(sentences):
    for sentence in sentences:
        label = classify_text(sentence)
        print(f"Text: {sentence}")
        print(f"Prediction: {label}\n")

In [9]:
sentences = [
    "J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.",
    "La vidéo était mauvaise. Je ne l'aime pas.",
    "Les immigrés vivent avec nous dans notre pays.",
    "Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée."
]
evaluate_sentences(sentences)

Text: J'ai adoré la vidéo, continuez comme ça. Je suis déjà abonné.
Prediction: []

Text: La vidéo était mauvaise. Je ne l'aime pas.
Prediction: []

Text: Les immigrés vivent avec nous dans notre pays.
Prediction: []

Text: Je ne suis pas d'accord avec ce que vous dites. Je pense qu'une autre solution peut être trouvée.
Prediction: []



## Custom Sentences: Difficult Examples

In [10]:
sentences = [
    "Je pense que les immigrés ne devraient pas exister dans notre pays.",    
    "Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.",
    "Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.",
    "Les hommes peuvent faire des travaux plus difficiles que les femmes.",
    "La présence des femmes dans le monde du travail est importante.",
    "Je n'ai pas du tout aimé la vidéo, mais bonne chance."
]
evaluate_sentences(sentences)

Text: Je pense que les immigrés ne devraient pas exister dans notre pays.
Prediction: []

Text: Je ne suis pas d'accord avec votre opinion, je pense que c'est ridicule.
Prediction: []

Text: Je ne suis pas d'accord avec ton opinion, je pense que c'est ridicule, mais je te respecte quand même.
Prediction: []

Text: Les hommes peuvent faire des travaux plus difficiles que les femmes.
Prediction: []

Text: La présence des femmes dans le monde du travail est importante.
Prediction: []

Text: Je n'ai pas du tout aimé la vidéo, mais bonne chance.
Prediction: []

